<a href="https://colab.research.google.com/github/kkfwees/cs230_2020/blob/master/evaluate_RCNN_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-_p06j9cy
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-_p06j9cy
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=267010 sha256=62beb02bc640a02a5582de0fec49499ae2cc36e82aa29fe93d5b7ae094ccc91d
  Stored in directory: /tmp/pip-ephem-wheel-cache-lof5jwfm/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.0
    Uninstalling pycocotools-2.0.0:
      Successfully uninstalled pycocotools-2.0.0


In [0]:
#get all needed resources
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import numpy as np
import torch
import torch.utils.data

In [3]:
# mount data to google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class SatelliteDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root,  "/content/drive/My Drive/CS230/data/bpop150k/unlabeled"))))
        self.imgs += list(sorted(os.listdir(os.path.join(root,  "/content/drive/My Drive/CS230/data/brural/unlabeled"))))
        self.imgs += list(sorted(os.listdir(os.path.join(root,  "/content/drive/My Drive/CS230/data/bpop1000k/unlabeled"))))
        self.imgs += list(sorted(os.listdir(os.path.join(root,  "/content/drive/My Drive/CS230/data/bpopunlmtd/unlabeled"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "/content/drive/My Drive/CS230/data/bpop150k/masks"))))
        self.masks += list(sorted(os.listdir(os.path.join(root, "/content/drive/My Drive/CS230/data/brural/masks"))))
        self.masks += list(sorted(os.listdir(os.path.join(root, "/content/drive/My Drive/CS230/data/bpop1000k/masks"))))
        self.masks += list(sorted(os.listdir(os.path.join(root, "/content/drive/My Drive/CS230/data/bpopunlmtd/masks"))))


    def __getitem__(self, idx):
        # load images and masks

        if self.imgs[idx].__contains__('bpop150k'):
          img_path = os.path.join(self.root,  "/content/drive/My Drive/CS230/data/bpop150k/unlabeled", self.imgs[idx])
          mask_path = os.path.join(self.root, "/content/drive/My Drive/CS230/data/bpop150k/masks", self.masks[idx])
        elif self.imgs[idx].__contains__('brural'):
          img_path = os.path.join(self.root,  "/content/drive/My Drive/CS230/data/brural/unlabeled", self.imgs[idx])
          mask_path = os.path.join(self.root, "/content/drive/My Drive/CS230/data/brural/masks", self.masks[idx])
        elif self.imgs[idx].__contains__('bpop1000k'):
          img_path = os.path.join(self.root,  "/content/drive/My Drive/CS230/data/bpop1000k/unlabeled", self.imgs[idx])
          mask_path = os.path.join(self.root, "/content/drive/My Drive/CS230/data/bpop1000k/masks", self.masks[idx])
        else:
          img_path = os.path.join(self.root,  "/content/drive/My Drive/CS230/data/bpopunlmtd/unlabeled", self.imgs[idx])
          mask_path = os.path.join(self.root, "/content/drive/My Drive/CS230/data/bpopunlmtd/masks", self.masks[idx])
        
        img = Image.open(img_path).convert("RGB")
        
  
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        # mask = Image.open(mask_path)
        #mask = np.array(mask)
        #print(mask.shape)
        mask = Image.open(mask_path).convert("L")
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)

        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            test_area = (ymax-ymin) * (xmax-xmin)
            if test_area > 0:
              boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [0]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [6]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8398 (delta 1), reused 1 (delta 0), pack-reused 8391
Receiving objects: 100% (8398/8398), 10.26 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (5778/5778), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be37608 version check against PyTorch's CUDA version


In [0]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

#CHANGE WEIGHTS HERE
# get pretrained weights
model.load_state_dict(torch.load('/content/drive/My Drive/CS230/parameters/RCNN_v3 (west_eu)/paramsrcnnv3westeu_model2bxnonlogis900_epoch2.pth'))

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


<All keys matched successfully>

In [0]:
dataset_human_label = SatelliteDataset('/content/drive/My Drive/CS230/data', get_transform(train=False))
torch.manual_seed(1)
indices = torch.randperm(len(dataset_human_label)).tolist()
dataset_human_label = torch.utils.data.Subset(dataset_human_label, indices[-250:])
num_dataset_human_label = len(dataset_human_label)


In [12]:
dev_filenames = os.listdir('/content/drive/My Drive/CS230/data/self_labeled/label_suggestions/IoU@25_threshold@20/final_masks')

num_dev = len(dev_filenames)

acc_per_img = np.zeros((num_dev,1))
recall_per_img = np.zeros((num_dev,1))

final_area = 0
overlap_area = 0
prd_area = 0
google_area = 0
overlap_google_area = 0
google_objects = 0
final_objects = 0

for k in range(num_dev):

  temp_idx = int(dev_filenames[k].split('.',2)[0])
  temp_img, temp_target = dataset_human_label[temp_idx]
  temp_prd_mask_img = temp_img.mul(255).permute(1, 2, 0).byte().numpy()

  model.eval()
  with torch.no_grad():
      prediction = model([temp_img.to(device)])

  #init image of predicted mask

  temp_prd_mask_img[:,:,0]= 0
  temp_prd_mask_img[:,:,1]= 0
  temp_prd_mask_img[:,:,2]= 0

  num_predicted_masks = len(prediction[0]['scores'])
  prd_mask = np.zeros((640,640))
  temp_prd_mask = np.zeros((640,640))

  #add predicted masks
  for i in range(0,num_predicted_masks):
    np.random.seed(3) 
    r_channel = np.random.uniform(low=0, high=255, size=num_predicted_masks).astype(int)
    np.random.seed(4) 
    g_channel = np.random.uniform(low=0, high=255, size=num_predicted_masks).astype(int)
    np.random.seed(5) 
    b_channel = np.random.uniform(low=0, high=255, size=num_predicted_masks).astype(int)
    if (prediction[0]['scores'][i] > 0.5):
      temp_prd_mask = np.asarray(prediction[0]['masks'][i, 0].cpu())
      temp_prd_mask[temp_prd_mask >= 0.5] = 1
      temp_prd_mask[temp_prd_mask < 0.5] = 0
      temp_prd_mask_img[:,:,0][temp_prd_mask==1] = r_channel[i]
      temp_prd_mask_img[:,:,1][temp_prd_mask==1] = g_channel[i]
      temp_prd_mask_img[:,:,2][temp_prd_mask==1] = b_channel[i]
      prd_mask[temp_prd_mask == 1] = 1 
      
  final_mask_path = '/content/drive/My Drive/CS230/data/self_labeled/label_suggestions/IoU@25_threshold@20/final_masks/'+dev_filenames[k]
  final_mask_img = Image.open(final_mask_path).convert("L")
  final_mask = np.array(final_mask_img)
  final_objects += len(np.unique(final_mask))-1
  final_mask[final_mask > 0 ] = 1


  acc_per_img[k] = np.sum(prd_mask * final_mask) / np.sum(prd_mask)
  recall_per_img[k]  = np.sum(prd_mask * final_mask) / np.sum(final_mask)
  final_area += np.sum(final_mask)
  prd_area += np.sum(prd_mask)
  overlap_area += np.sum(prd_mask * final_mask)

  google_mask = np.array(temp_target['masks'])
  google_area += np.sum(google_mask)
  overlap_google_area += np.sum(google_mask * final_mask)
  google_objects += len(temp_target['labels'])


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars


In [13]:
overall_recall = overlap_area / final_area
overall_acc = overlap_area / prd_area
overall_f1 = 2*(overall_recall * overall_acc)/(overall_recall + overall_acc)
print(overall_recall)
print(overall_acc)
print(overall_f1)
print(final_area)

0.5977355144817518
0.7568170141241489
0.667934831248848
7035682.0


In [14]:
google_recall = overlap_google_area / final_area
google_acc = overlap_google_area / google_area
google_f1 = 2*(google_recall * google_acc)/(google_recall + google_acc)
print(google_recall)
print(google_acc)
print(google_f1)
print(google_area)

0.29804416970522546
0.8979865208495171
0.44754645365988377
2335162.0
